<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Optimized%20SWAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
import timeit
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Embedding, Dense, concatenate, Dropout, Flatten, Activation
from keras import backend as K
from keras.utils import get_custom_objects, to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import load_iris, load_digits, load_wine, load_diabetes, load_breast_cancer, fetch_olivetti_faces, load_linnerud



In [48]:

# Set random seed
np.random.seed(110)
# Defining new activation functions
def define_activation_functions():
    """
    Define custom activation functions and update the custom objects dictionary.
    """
    # Define activation functions and their respective names
    activation_functions = [
        ('X_1', lambda x: K.pow(x, 1)),
        ('X_2', lambda x: K.pow(x, 2) / 2),
        ('X_2_', lambda x: K.pow(x, 2) / 24),
        ('X_2__', lambda x: K.pow(x, 2) / 720),
        ('X_2___', lambda x: K.pow(x, 2) / 40320),
        ('X_3', lambda x: K.pow(x, 3) / 6),
        ('X_4', lambda x: K.pow(x, 4) / 24),
        ('X_5', lambda x: K.pow(x, 5) / 120),
        ('X_6', lambda x: K.pow(x, 6) / 720),
    ]

    # Update the custom objects dictionary with the defined activation functions
    for name, function in activation_functions:
        get_custom_objects().update({name: Activation(function)})

# Define and register the custom activation functions
define_activation_functions()

def create_optimized_model(input_dim,  hidden_dim , output_dim , metrics_ ):
    """
    Create an optimized SWAG model with a custom architecture.

    :param input_dim: int, dimension of the input data
    :param output_dim: int, dimension of the output data
    :param hidden_dim: int, hidden layer dimension, default is 50
    :return: Model, a compiled Keras model
    """

    # Define input layer
    input_layer = Input(shape=(input_dim,))

    # First layer with custom activations
    layer_1_x1 = Dense(hidden_dim, activation='X_1')(input_layer)
    layer_1_x2 = Dense(hidden_dim, activation='X_2')(input_layer)
    concat_first_layer = concatenate([layer_1_x1, layer_1_x2])

    # Second layer with custom activations
    layer_x3_x4 = Dense(hidden_dim, activation='X_2_')(concat_first_layer)

    # Third layer with custom activations
    layer_x5_x6 = Dense(hidden_dim, activation='X_2__')(layer_x3_x4)



    # Concatenate all layers
    concat_second_layer = concatenate([layer_1_x1, layer_1_x2, concat_first_layer,
                                       layer_x3_x4, layer_x5_x6])

    # Output layer for the concatenated layers
    output_first_layer = Dense(hidden_dim, activation='linear')(concat_second_layer)

    # Final output layer
    output_layer = Dense(output_dim, activation='linear')(output_first_layer)

    # Create and compile the model
    model = Model(input_layer, output_layer)
    # Define the optimizer
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=metrics_)
    # model.compile(loss='mean_squared_error', optimizer='adam')

    return model

In [49]:
# optimized_model = create_optimized_model(784,784, 10)

# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = (x_train + 10) / 300.0, (x_test + 10) / 300.0
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Create and train the model
optimized_model = create_optimized_model(784, 500, 10 , 'accuracy')
batch_size = 100
epochs = 2
start = timeit.default_timer()
history = optimized_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

Epoch 1/2
600/600 [==============================] - 5s 5ms/step - loss: 0.0414 - accuracy: 0.8390 - val_loss: 0.0251 - val_accuracy: 0.9128
Epoch 2/2
600/600 [==============================] - 3s 4ms/step - loss: 0.0207 - accuracy: 0.9360 - val_loss: 0.0206 - val_accuracy: 0.9266
Training time: 12.301850872999239 seconds


In [60]:


iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)


num_features = X_train.shape[1]
num_classes = y_train.shape[1]



# Create and train the model
model = create_optimized_model(num_features, 10, num_classes, 'accuracy')
batch_size = 10
epochs = 10
start = timeit.default_timer()
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")

Epoch 1/10
12/12 [==============================] - 2s 21ms/step - loss: 0.4406 - accuracy: 0.7667 - val_loss: 0.2354 - val_accuracy: 0.8667
Epoch 2/10
12/12 [==============================] - 0s 8ms/step - loss: 0.1724 - accuracy: 0.8417 - val_loss: 0.1127 - val_accuracy: 0.8667
Epoch 3/10
12/12 [==============================] - 0s 7ms/step - loss: 0.1187 - accuracy: 0.8833 - val_loss: 0.0966 - val_accuracy: 0.8667
Epoch 4/10
12/12 [==============================] - 0s 6ms/step - loss: 0.0987 - accuracy: 0.9000 - val_loss: 0.0889 - val_accuracy: 0.9000
Epoch 5/10
12/12 [==============================] - 0s 6ms/step - loss: 0.0897 - accuracy: 0.9083 - val_loss: 0.0879 - val_accuracy: 0.9333
Epoch 6/10
12/12 [==============================] - 0s 7ms/step - loss: 0.0848 - accuracy: 0.9000 - val_loss: 0.0828 - val_accuracy: 0.9333
Epoch 7/10
12/12 [==============================] - 0s 6ms/step - loss: 0.0806 - accuracy: 0.9167 - val_loss: 0.0776 - val_accuracy: 0.9333
Epoch 8/10
12/12 [=

In [62]:




# Load and preprocess the dataset
digits = load_digits()
X = digits.data
y = to_categorical(digits.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 200, num_classes, 'accuracy')
batch_size = 10
epochs = 5

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


64
Epoch 1/5
144/144 [==============================] - 3s 4ms/step - loss: 0.0520 - accuracy: 0.8497
Epoch 2/5
144/144 [==============================] - 1s 4ms/step - loss: 0.0212 - accuracy: 0.9659
Epoch 3/5
144/144 [==============================] - 1s 4ms/step - loss: 0.0160 - accuracy: 0.9861
Epoch 4/5
144/144 [==============================] - 1s 4ms/step - loss: 0.0137 - accuracy: 0.9896
Epoch 5/5
12/12 [==============================] - 0s 3ms/step - loss: 0.0119 - accuracy: 0.9972
Accuracy: 0.9972222447395325


In [64]:


# Load and preprocess the dataset
wine = load_wine()
X = wine.data
y = to_categorical(wine.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, num_classes, 'accuracy')
batch_size = 10
epochs = 5

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")

13
Epoch 1/5
15/15 [==============================] - 2s 5ms/step - loss: 0.2782 - accuracy: 0.3451
Epoch 2/5
15/15 [==============================] - 0s 4ms/step - loss: 0.1720 - accuracy: 0.7042
Epoch 3/5
15/15 [==============================] - 0s 5ms/step - loss: 0.1109 - accuracy: 0.8873
Epoch 4/5
15/15 [==============================] - 0s 5ms/step - loss: 0.0808 - accuracy: 0.9437
Epoch 5/5
2/2 [==============================] - 0s 9ms/step - loss: 0.0532 - accuracy: 0.9722
Accuracy: 0.9722222089767456


In [66]:


# Load and preprocess the dataset
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'accuracy')
batch_size = 10
epochs = 5

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


30
Epoch 1/5
46/46 [==============================] - 3s 4ms/step - loss: 0.2308 - accuracy: 0.6593
Epoch 2/5
46/46 [==============================] - 0s 4ms/step - loss: 0.1282 - accuracy: 0.8593
Epoch 3/5
46/46 [==============================] - 0s 5ms/step - loss: 0.1187 - accuracy: 0.8901
Epoch 4/5
46/46 [==============================] - 0s 4ms/step - loss: 0.0965 - accuracy: 0.9165
Epoch 5/5
4/4 [==============================] - 0s 5ms/step - loss: 0.0925 - accuracy: 0.9386
Accuracy: 0.9385964870452881


In [68]:




# Load Boston Housing data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the model
input_dim = X_train.shape[1]
optimized_model = create_optimized_model(input_dim, input_dim , 1 , 'mae') # input_dim and hidden_dim
batch_size = 5
epochs = 30
start = timeit.default_timer()
history = optimized_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_test, y_test))
end = timeit.default_timer()

# Training time
print(f"Training time: {end - start} seconds")


Epoch 1/30
81/81 [==============================] - 3s 7ms/step - loss: 382.4527 - mae: 16.4929 - val_loss: 195.3077 - val_mae: 11.2270
Epoch 2/30
81/81 [==============================] - 1s 8ms/step - loss: 134.5626 - mae: 9.0382 - val_loss: 109.9852 - val_mae: 7.8617
Epoch 3/30
81/81 [==============================] - 1s 7ms/step - loss: 81.2563 - mae: 6.6544 - val_loss: 77.1532 - val_mae: 6.5770
Epoch 4/30
81/81 [==============================] - 1s 7ms/step - loss: 63.9724 - mae: 5.8040 - val_loss: 66.7118 - val_mae: 6.0760
Epoch 5/30
81/81 [==============================] - 1s 8ms/step - loss: 56.6062 - mae: 5.4431 - val_loss: 59.9075 - val_mae: 5.8186
Epoch 6/30
81/81 [==============================] - 1s 8ms/step - loss: 51.0353 - mae: 5.2252 - val_loss: 54.2574 - val_mae: 5.3945
Epoch 7/30
81/81 [==============================] - 1s 8ms/step - loss: 46.4667 - mae: 4.9456 - val_loss: 49.4718 - val_mae: 5.1582
Epoch 8/30
81/81 [==============================] - 0s 5ms/step - loss

In [69]:



# Load the dataset
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features to be between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'mae')
batch_size = 10
epochs = 100

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


10
Epoch 1/100
36/36 [==============================] - 2s 4ms/step - loss: 29246.2324 - mae: 152.4018
Epoch 2/100
36/36 [==============================] - 0s 4ms/step - loss: 26537.0234 - mae: 143.9060
Epoch 3/100
36/36 [==============================] - 0s 4ms/step - loss: 17714.8398 - mae: 110.8377
Epoch 4/100
36/36 [==============================] - 0s 4ms/step - loss: 5976.2329 - mae: 61.3968
Epoch 5/100
36/36 [==============================] - 0s 4ms/step - loss: 5028.2324 - mae: 56.4191
Epoch 6/100
36/36 [==============================] - 0s 4ms/step - loss: 4813.5527 - mae: 55.6264
Epoch 7/100
36/36 [==============================] - 0s 4ms/step - loss: 4495.3188 - mae: 54.1489
Epoch 8/100
36/36 [==============================] - 0s 4ms/step - loss: 4355.5532 - mae: 53.1489
Epoch 9/100
36/36 [==============================] - 0s 4ms/step - loss: 4260.6431 - mae: 52.5106
Epoch 10/100
36/36 [==============================] - 0s 5ms/step - loss: 4065.5613 - mae: 51.6973
Epoch 11/1

In [70]:


# Load and preprocess the dataset
faces = fetch_olivetti_faces()
X = faces.images
y = to_categorical(faces.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reshape((-1, 64*64))
X_test = X_test.reshape((-1, 64*64))

# Use MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
num_classes = y.shape[1]

print(num_classes)

# Create and train the model
model = create_optimized_model(num_features, 90, num_classes, 'accuracy')
batch_size = 10
epochs = 20

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")


40
Epoch 1/20
32/32 [==============================] - 2s 4ms/step - loss: 4.1380 - accuracy: 0.0219
Epoch 2/20
32/32 [==============================] - 0s 4ms/step - loss: 0.2143 - accuracy: 0.0281
Epoch 3/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0693 - accuracy: 0.0719
Epoch 4/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0498 - accuracy: 0.1219
Epoch 5/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0415 - accuracy: 0.2344
Epoch 6/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0387 - accuracy: 0.2844
Epoch 7/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0382 - accuracy: 0.3562
Epoch 8/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0331 - accuracy: 0.4938
Epoch 9/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0300 - accuracy: 0.5375
Epoch 10/20
32/32 [==============================] - 0s 4ms/step - loss: 0.0298 - accuracy: 0.5969
Epoch 11/20
32/3

In [71]:


# Load and preprocess the dataset
linnerud = load_linnerud()
X = linnerud.data
y = linnerud.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Using MinMaxScaler to scale the data between 0 and 1
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

num_features = X_train.shape[1]
# num_classes = y_train.shape[1]

print(num_features)

# Create and train the model
model = create_optimized_model(num_features, 20, 1, 'mae')
batch_size = 10
epochs = 100

# Train the model
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy}")

3
Epoch 1/100
2/2 [==============================] - 2s 11ms/step - loss: 12104.5264 - mae: 89.2977
Epoch 2/100
2/2 [==============================] - 0s 9ms/step - loss: 12082.5508 - mae: 89.1712
Epoch 3/100
2/2 [==============================] - 0s 8ms/step - loss: 12063.2148 - mae: 89.0591
Epoch 4/100
2/2 [==============================] - 0s 8ms/step - loss: 12041.7520 - mae: 88.9352
Epoch 5/100
2/2 [==============================] - 0s 8ms/step - loss: 12019.1719 - mae: 88.8037
Epoch 6/100
2/2 [==============================] - 0s 8ms/step - loss: 11998.2402 - mae: 88.6818
Epoch 7/100
2/2 [==============================] - 0s 8ms/step - loss: 11975.5566 - mae: 88.5502
Epoch 8/100
2/2 [==============================] - 0s 8ms/step - loss: 11952.7979 - mae: 88.4178
Epoch 9/100
2/2 [==============================] - 0s 13ms/step - loss: 11927.3809 - mae: 88.2686
Epoch 10/100
2/2 [==============================] - 0s 8ms/step - loss: 11903.5488 - mae: 88.1303
Epoch 11/100
2/2 [=======